# Getting started

Once you've chosen your scenario, download the data from [the Iowa website](https://data.iowa.gov/Economy/Iowa-Liquor-Sales/m3tr-qhgy) in csv format. Start by loading the data with pandas. You may need to parse the date columns appropriately.

In [2]:
% matplotlib inline
import datetime
import numpy as np
import pandas as pd

## Load the data into a DataFrame
# pd.read_csv()
df = pd.read_csv("/Users/russellsasaki/Downloads/Iowa_Liquor_sales_sample_10pct.csv")
print df.columns
df.head()
## Transform the dates if needed, e.g.
# df["Date"] = pd.to_datetime(df["Date"], format="%m-%d-%y")

Index([u'Date', u'Store Number', u'City', u'Zip Code', u'County Number',
       u'County', u'Category', u'Category Name', u'Vendor Number',
       u'Item Number', u'Item Description', u'Bottle Volume (ml)',
       u'State Bottle Cost', u'State Bottle Retail', u'Bottles Sold',
       u'Sale (Dollars)', u'Volume Sold (Liters)', u'Volume Sold (Gallons)'],
      dtype='object')


,Date,Store Number,City,Zip Code,County Number,County,Category,Category Name,Vendor Number,Item Number,Item Description,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,11/04/2015,3717,SUMNER,50674,9.0,Bremer,1051100.0,APRICOT BRANDIES,55,54436,Mr. Boston Apricot Brandy,750,$4.50,$6.75,12,$81.00,9.0,2.38
1,03/02/2016,2614,DAVENPORT,52807,82.0,Scott,1011100.0,BLENDED WHISKIES,395,27605,Tin Cup,750,$13.75,$20.63,2,$41.26,1.5,0.40
2,02/11/2016,2106,CEDAR FALLS,50613,7.0,Black Hawk,1011200.0,STRAIGHT BOURBON WHISKIES,65,19067,Jim Beam,1000,$12.59,$18.89,24,$453.36,24.0,6.34
3,02/03/2016,2501,AMES,50010,85.0,Story,1071100.0,AMERICAN COCKTAILS,395,59154,1800 Ultimate Margarita,1750,$9.50,$14.25,6,$85.50,10.5,2.77
4,08/18/2015,3654,BELMOND,50421,99.0,Wright,1031080.0,VODKA 80 PROOF,297,35918,Five O'clock Vodka,1750,$7.20,$10.80,12,$129.60,21.0,5.55


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270955 entries, 0 to 270954
Data columns (total 18 columns):
Date                     270955 non-null object
Store Number             270955 non-null int64
City                     270955 non-null object
Zip Code                 270955 non-null object
County Number            269878 non-null float64
County                   269878 non-null object
Category                 270887 non-null float64
Category Name            270323 non-null object
Vendor Number            270955 non-null int64
Item Number              270955 non-null int64
Item Description         270955 non-null object
Bottle Volume (ml)       270955 non-null int64
State Bottle Cost        270955 non-null object
State Bottle Retail      270955 non-null object
Bottles Sold             270955 non-null int64
Sale (Dollars)           270955 non-null object
Volume Sold (Liters)     270955 non-null float64
Volume Sold (Gallons)    270955 non-null float64
dtypes: float64(4), int64(

# Mine the data
Now you are ready to compute the variables you will use for your regression from the data. For example, you may want to
compute total sales per store from Jan to March of 2015, mean price per bottle, etc. Refer to the readme for more ideas appropriate to your scenario.

Pandas is your friend for this task. Take a look at the operations [here](http://pandas.pydata.org/pandas-docs/stable/groupby.html) for ideas on how to make the best use of pandas and feel free to search for blog and Stack Overflow posts to help you group data by certain variables and compute sums, means, etc. You may find it useful to create a new data frame to house this summary data.

In [5]:
# Remove redundant columns
del df["Category Name"]
del df["Item Description"]
del df["County"]


In [6]:
# Remove $ from certain columns
# Remove $ from certain columns
cols = ["State Bottle Cost", "State Bottle Retail", "Sale (Dollars)"]

for col in cols:
    df[col] = df[col].apply(lambda x: float(x[1:]))

In [7]:
mystring = "$500"
mystring[1:]

'500'

In [8]:
# Convert dates
df["Date"] = pd.to_datetime(df["Date"], format="%m/%d/%Y")

In [9]:
# Drop or replace bad values
df.dropna(inplace=True)


# Convert integers
df["County Number"] = df["County Number"].astype(int)
df["Category"] = df["Category"].astype(int)


In [10]:
# Determine which stores were open all of 2015
# Find the first and last sales date.
dates = df.groupby(by=["Store Number"], as_index=False)


dates = dates.agg({"Date": [np.min, np.max]})
dates.columns=[''.join(col).strip() for col in dates.columns.values]
dates.head()

# Filter out stores that opened or closed throughout the year
lower_cutoff=pd.Timestamp('20150101')
upper_cutoff=pd.Timestamp('20151231')

mask = (dates["Dateamin"] < lower_cutoff) & (dates['Dateamax'] 
                                              > upper_cutoff)

good_stores = dates[mask]["Store Number"]

In [11]:
good_stores

Series([], Name: Store Number, dtype: int64)

In [9]:
df.head()

,Date,Store Number,City,Zip Code,County Number,Category,Vendor Number,Item Number,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,2015-11-04,3717,SUMNER,50674,9,1051100,55,54436,750,4.50,6.75,12,81.00,9.0,2.38
1,2016-03-02,2614,DAVENPORT,52807,82,1011100,395,27605,750,13.75,20.63,2,41.26,1.5,0.40
2,2016-02-11,2106,CEDAR FALLS,50613,7,1011200,65,19067,1000,12.59,18.89,24,453.36,24.0,6.34
3,2016-02-03,2501,AMES,50010,85,1071100,395,59154,1750,9.50,14.25,6,85.50,10.5,2.77
4,2015-08-18,3654,BELMOND,50421,99,1031080,297,35918,1750,7.20,10.80,12,129.60,21.0,5.55


In [41]:
# Margin and Price per liter
df['Margin']= (df['State Bottle Retail']*df['Bottles Sold'])-(df['State Bottle Cost']* df['Bottles Sold'])
df['Price per Liter']= df['Sale (Dollars)']/ df['Volume Sold (Liters)']
df.head()

,Date,Store Number,City,Zip Code,County Number,Category,Vendor Number,Item Number,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),Margin,Price per Liter
22111,2015-01-08,2106,CEDAR FALLS,50613,7,1062200,35,43127,1000,9.50,14.25,36,513.00,36.0,9.51,171.00,14.25
47659,2015-01-08,2106,CEDAR FALLS,50613,7,1022100,410,88296,750,27.00,40.50,12,486.00,9.0,2.38,162.00,54.00
65214,2015-01-08,2106,CEDAR FALLS,50613,7,1062310,259,43026,750,5.78,8.67,12,104.04,9.0,2.38,34.68,11.56
66346,2015-01-08,2106,CEDAR FALLS,50613,7,1081200,305,73055,750,12.50,18.75,12,225.00,9.0,2.38,75.00,25.00
80570,2015-01-08,2106,CEDAR FALLS,50613,7,1062200,434,46351,1000,4.34,6.51,12,78.12,12.0,3.17,26.04,6.51


In [184]:
##### Sales per store, 2015

# Filter by our start and end dates
df.sort_values(by=["Store Number", "Date"], inplace=True)
start_date = pd.Timestamp("20150101")
end_date = pd.Timestamp("20151231")
mask = (df['Date'] >= start_date) & (df['Date'] <= end_date)
sales = df[mask]

# Group by store name
sales = sales.groupby(by=["Store Number", 'Zip Code', 'City','County Number', 'Bottles Sold'], as_index=False)

sales = sales.agg({ "Sale (Dollars)": [np.sum, np.mean],
                   "Volume Sold (Liters)": [np.sum, np.mean],
                   "Margin": np.mean,"Margin": np.sum,
                   "Price per Liter": np.mean
                   })
# Collapse the column indices
sales.columns = [' '.join(col).strip() for col in sales.columns.values]
# Rename columns

# Quick check
sales.head()


,Store Number,Zip Code,City,County Number,Bottles Sold,Price per Liter mean,Margin sum,Sale (Dollars) sum,Sale (Dollars) mean,Volume Sold (Liters) sum,Volume Sold (Liters) mean
0,2106,50613,CEDAR FALLS,7,1,30.775238,103.06,309.14,22.081429,9.10,0.6500
1,2106,50613,CEDAR FALLS,7,2,25.923611,317.54,946.88,39.453333,37.50,1.5625
2,2106,50613,CEDAR FALLS,7,3,17.957333,147.42,442.20,44.220000,25.50,2.5500
3,2106,50613,CEDAR FALLS,7,4,32.468333,134.36,403.00,100.750000,14.00,3.5000
4,2106,50613,CEDAR FALLS,7,5,20.000000,25.00,75.00,75.000000,3.75,3.7500


In [181]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12397 entries, 0 to 12396
Data columns (total 11 columns):
Store Number                 12397 non-null int64
Zip Code                     12397 non-null object
City                         12397 non-null object
County Number                12397 non-null int64
Bottles Sold                 12397 non-null int64
Price per Liter mean         12397 non-null float64
Margin mean                  12397 non-null float64
Sale (Dollars) sum           12397 non-null float64
Sale (Dollars) mean          12397 non-null float64
Volume Sold (Liters) sum     12397 non-null float64
Volume Sold (Liters) mean    12397 non-null float64
dtypes: float64(6), int64(3), object(2)
memory usage: 1.1+ MB


In [75]:
store_margin=sales.groupby(['Store Number']).agg({'Margin sum':np.sum})

In [76]:
store_margin.sort_values('Margin sum',axis=0, ascending=False)

,Margin sum
Store Number,
2633,333816.91
4829,293196.99
3385,147884.62
2512,140143.46
3420,117593.34
3952,111230.21
3814,83004.48
3354,80244.69
3773,77525.08


In [77]:
zip_code_margin=sales.groupby(['Zip Code']).agg({'Margin sum':np.sum})

In [78]:
zip_code_margin.sort_values('Margin sum', axis=0, ascending=False)

,Margin sum
Zip Code,
50320,357269.15
50314,357011.44
52402,328228.12
52240,292839.51
50010,258178.25
52807,199324.51
50311,198192.62
52722,189199.99
51501,184031.18


In [96]:
city_margin=sales.groupby(['City','Zip Code']).agg({'Margin sum':np.sum})

In [97]:
city_margin.sort_values('Margin sum', axis=0, ascending=False)

Margin sum
City            Zip Code            
DES MOINES      50320      357269.15
                50314      357011.44
CEDAR RAPIDS    52402      328228.12
IOWA CITY       52240      292839.51
AMES            50010      258178.25
DAVENPORT       52807      199324.51
BETTENDORF      52722      189199.99
COUNCIL BLUFFS  51501      184031.18
WEST DES MOINES 50266      181651.37
CORALVILLE      52241      176377.35
CEDAR FALLS     50613      172650.45
DUBUQUE         52001      170399.03
WINDSOR HEIGHTS 50311      161565.45
ANKENY          50021      140000.28
WEST DES MOINES 50265      139732.52
MASON CITY      50401      132844.28
COUNCIL BLUFFS  51503      131572.90
WATERLOO        50703      131535.86
CEDAR RAPIDS    52404      127670.82
FORT DODGE      50501      127076.09
DAVENPORT       52804      125568.93
SIOUX CITY      51106      125332.95
WATERLOO        50702      122440.43
DES MOINES      50317       96834.66
                50315       94563.42
BURLINGTON      52601       92224.92
URBANDALE       50322       91827.81
MARSHALLTOWN    50158       89703.70
MUSCATINE       52761       89129.64
CLINTON         52732       88658.75
...                              ...
WALL LAKE       51466         573.12
NEW SHARON      50207         552.21
VAN METER       50261         551.70
WELLSBURG       50680         528.54
ROLFE           50581         522.30
MEDIAPOLIS      52405         510.88
CLIVE           50322         510.44
AURELIA         51005         508.26
GILMORE CITY    50541         492.00
KELLOG          50136         491.14
CAMBRIDGE       50046         478.32
SCRANTON        51462         457.10
NORTH ENGLISH   52316         434.08
WALFORD         52351         387.42
SCHALLER        51053         352.27
GOLDFIELD       50542         347.88
LATIMER         50452         345.54
VILLISCA        50864         341.32
WHEATLAND       52777         299.63
MELBOURNE       50162         296.74
DANVILLE        52623         283.98
LOVILIA         50150         271.00
EVERLY          51338         191.64
MINDEN          51553         191.16
MERRILL         51038         177.32
STANWOOD        52337         157.56
GRISWOLD        51535         140.28
DELHI           52223          74.99
LOHRVILLE       51453          46.53
ROBINS          52328          30.00

[436 rows x 1 columns]

In [112]:
zips_in_city=sales.groupby(['City', 'Zip Code']).agg({'Zip Code':'count'})
zips_in_city.sort_values('Zip Code', axis=0, ascending=False)

,,Zip Code
City,Zip Code,
CEDAR RAPIDS,52402,28
COUNCIL BLUFFS,51501,26
CEDAR RAPIDS,52404,24
AMES,50010,21
IOWA CITY,52240,20
DES MOINES,50317,18
WATERLOO,50703,16
DUBUQUE,52001,16
CEDAR FALLS,50613,16


In [108]:
sales['Zip Code'].value_counts()

52402    28
51501    26
52404    24
50010    21
52240    20
50317    20
50322    19
52001    18
50266    16
50703    16
50702    16
50613    16
51503    15
50265    15
52761    15
50501    15
50701    14
50401    13
52241    13
50158    12
52722    12
52405    12
50021    12
52601    12
51106    12
50315    11
52501    11
52302    10
50588    10
52804    10
         ..
52328     1
51553     1
50048     1
50452     1
50635     1
51535     1
50841     1
52151     1
51050     1
52223     1
51001     1
51002     1
51551     1
51004     1
51005     1
52087     1
52084     1
51559     1
51358     1
50535     1
51355     1
50225     1
52751     1
51058     1
50228     1
52351     1
52316     1
52352     1
52356     1
52571     1
Name: Zip Code, dtype: int64

In [122]:
margin_sum_max=sales.groupby(['City', 'Zip Code', 'Store Number']).agg({'Margin sum':np.sum})
margin_sum_max.sort_values('Margin sum', axis=0, ascending=False)

Margin sum
City            Zip Code Store Number            
DES MOINES      50320    2633           333816.91
                50314    4829           293196.99
CEDAR RAPIDS    52402    3385           147884.62
IOWA CITY       52240    2512           140143.46
WINDSOR HEIGHTS 50311    3420           117593.34
BETTENDORF      52722    3952           111230.21
WEST DES MOINES 50266    3814            83004.48
DAVENPORT       52807    3354            80244.69
CEDAR RAPIDS    52401    3773            77525.08
CORALVILLE      52241    2670            75853.72
CARROLL         51401    2593            64197.45
SIOUX CITY      51106    3447            62683.22
COUNCIL BLUFFS  51503    2629            61636.70
MOUNT VERNON    52314    5102            61504.64
DAVENPORT       52804    2625            60794.18
AMES            50010    3524            60528.54
URBANDALE       50322    2663            58512.28
SIOUX CITY      51103    3820            57650.18
DUBUQUE         52001    4167            55756.18
WEST DES MOINES 50265    2648            52870.35
DES MOINES      50321    2561            52844.61
CLINTON         52732    2616            51472.43
CORALVILLE      52241    4677            50517.91
ANKENY          50021    2502            50345.31
AMES            50010    2500            49801.48
CEDAR FALLS     50613    2106            48838.08
COUNCIL BLUFFS  51501    4312            48493.09
AMES            50010    2501            48430.73
WEST DES MOINES 50266    2619            48105.29
BURLINGTON      52601    2506            47925.66
...                                           ...
URBANDALE       50322    4954              190.44
MERRILL         51038    4985              177.32
ADEL            50003    4929              176.16
WATERLOO        50702    5152              173.68
MASON CITY      50401    5212              168.28
GRUNDY CENTER   50638    5206              167.02
STORM LAKE      50588    4737              164.64
STANWOOD        52337    5020              157.56
COUNCIL BLUFFS  51501    5161              154.55
DUBUQUE         52003    4961              150.09
GRISWOLD        51535    4990              140.28
CARROLL         51401    4776              125.64
WAPELLO         52653    4121              125.04
WEST LIBERTY    52776    5178              112.19
STORM LAKE      50588    4778              102.00
GUTHRIE CENTER  50115    5191              101.34
WEST UNION      52175    5186               99.72
DUBUQUE         52001    5201               96.65
MUSCATINE       52761    4834               90.36
DELHI           52223    5175               74.99
MONTROSE        52639    5213               60.00
COUNCIL BLUFFS  51501    5056               58.24
LOHRVILLE       51453    5193               46.53
BELLE PLAINE    52208    4059               35.15
ROBINS          52328    5192               30.00
IOWA CITY       52245    5053               22.50
LE CLAIRE       52753    5215               18.68
URBANDALE       50322    4939               17.81
DES MOINES      50313    4567               15.06
GLIDDEN         51443    5205               13.34

[1373 rows x 1 columns]

In [126]:
county_max_margin=sales.groupby(['County Number','City', 'Zip Code', 'Store Number']).agg({'Margin sum':np.sum
                                                                                           })
county_max_margin.sort_values('Margin sum', axis=0, ascending=False)

Margin sum
County Number City            Zip Code Store Number            
77            DES MOINES      50320    2633           333816.91
                              50314    4829           293196.99
57            CEDAR RAPIDS    52402    3385           147884.62
52            IOWA CITY       52240    2512           140143.46
77            WINDSOR HEIGHTS 50311    3420           117593.34
82            BETTENDORF      52722    3952           111230.21
25            WEST DES MOINES 50266    3814            83004.48
82            DAVENPORT       52807    3354            80244.69
57            CEDAR RAPIDS    52401    3773            77525.08
52            CORALVILLE      52241    2670            75853.72
14            CARROLL         51401    2593            64197.45
97            SIOUX CITY      51106    3447            62683.22
78            COUNCIL BLUFFS  51503    2629            61636.70
57            MOUNT VERNON    52314    5102            61504.64
82            DAVENPORT       52804    2625            60794.18
85            AMES            50010    3524            60528.54
77            URBANDALE       50322    2663            58512.28
97            SIOUX CITY      51103    3820            57650.18
31            DUBUQUE         52001    4167            55756.18
77            WEST DES MOINES 50265    2648            52870.35
              DES MOINES      50321    2561            52844.61
23            CLINTON         52732    2616            51472.43
52            CORALVILLE      52241    4677            50517.91
77            ANKENY          50021    2502            50345.31
85            AMES            50010    2500            49801.48
7             CEDAR FALLS     50613    2106            48838.08
78            COUNCIL BLUFFS  51501    4312            48493.09
85            AMES            50010    2501            48430.73
77            WEST DES MOINES 50266    2619            48105.29
29            BURLINGTON      52601    2506            47925.66
...                                                         ...
77            URBANDALE       50322    4954              190.44
75            MERRILL         51038    4985              177.32
25            ADEL            50003    4929              176.16
7             WATERLOO        50702    5152              173.68
17            MASON CITY      50401    5212              168.28
38            GRUNDY CENTER   50638    5206              167.02
11            STORM LAKE      50588    4737              164.64
16            STANWOOD        52337    5020              157.56
78            COUNCIL BLUFFS  51501    5161              154.55
31            DUBUQUE         52003    4961              150.09
15            GRISWOLD        51535    4990              140.28
14            CARROLL         51401    4776              125.64
58            WAPELLO         52653    4121              125.04
70            WEST LIBERTY    52776    5178              112.19
11            STORM LAKE      50588    4778              102.00
39            GUTHRIE CENTER  50115    5191              101.34
33            WEST UNION      52175    5186               99.72
31            DUBUQUE         52001    5201               96.65
70            MUSCATINE       52761    4834               90.36
28            DELHI           52223    5175               74.99
56            MONTROSE        52639    5213               60.00
78            COUNCIL BLUFFS  51501    5056               58.24
13            LOHRVILLE       51453    5193               46.53
6             BELLE PLAINE    52208    4059               35.15
57            ROBINS          52328    5192               30.00
52            IOWA CITY       52245    5053               22.50
82            LE CLAIRE       52753    5215               18.68
77            URBANDALE       50322    4939               17.81
              DES MOINES      50313    4567               15.06
14            GLIDDEN         51443    5205               13.34

[1373 rows x 1 colum

In [167]:
county_max_margin_per_L=sales.groupby(['County Number','City', 'Zip Code', 'Store Number']).agg({'Margin sum':np.sum,
                                                                                           'Price per Liter mean':np.sum})
county_max_margin_per_L.sort_values('Margin sum', axis=0, ascending=False).head(25)

Price per Liter mean  \
County Number City            Zip Code Store Number                         
77            DES MOINES      50320    2633                     21.019238   
                              50314    4829                     21.449783   
57            CEDAR RAPIDS    52402    3385                     19.078256   
52            IOWA CITY       52240    2512                     19.783300   
77            WINDSOR HEIGHTS 50311    3420                     19.146492   
82            BETTENDORF      52722    3952                     23.168556   
25            WEST DES MOINES 50266    3814                     27.860845   
82            DAVENPORT       52807    3354                     19.001827   
57            CEDAR RAPIDS    52401    3773                     24.662500   
52            CORALVILLE      52241    2670                     19.260802   
14            CARROLL         51401    2593                     18.432430   
97            SIOUX CITY      51106    3447                     17.730097   
78            COUNCIL BLUFFS  51503    2629                     19.140218   
57            MOUNT VERNON    52314    5102                     20.717103   
82            DAVENPORT       52804    2625                     18.676541   
85            AMES            50010    3524                     17.985769   
77            URBANDALE       50322    2663                     20.604345   
97            SIOUX CITY      51103    3820                     20.524943   
31            DUBUQUE         52001    4167                     20.730475   
77            WEST DES MOINES 50265    2648                     22.403684   
              DES MOINES      50321    2561                     20.117237   
23            CLINTON         52732    2616                     18.648622   
52            CORALVILLE      52241    4677                     27.704054   
77            ANKENY          50021    2502                     20.320363   
85            AMES            50010    2500                     18.988761   

                                                     Margin sum  
County Number City            Zip Code Store Number              
77            DES MOINES      50320    2633           333816.91  
                              50314    4829           293196.99  
57            CEDAR RAPIDS    52402    3385           147884.62  
52            IOWA CITY       52240    2512           140143.46  
77            WINDSOR HEIGHTS 50311    3420           117593.34  
82            BETTENDORF      52722    3952           111230.21  
25            WEST DES MOINES 50266    3814            83004.48  
82            DAVENPORT       52807    3354            80244.69  
57            CEDAR RAPIDS    52401    3773            77525.08  
52            CORALVILLE      52241    2670            75853.72  
14            CARROLL         51401    2593            64197.45  
97            SIOUX CITY      51106    3447            62683.22  
78            COUNCIL BLUFFS  51503    2629            61636.70  
57            MOUNT VERNON    52314    5102            61504.64  
82            DAVENPORT       52804    2625            60794.18  
85            AMES            50010    3524            60528.54  
77            URBANDALE       50322    2663            58512.28  
97            SIOUX CITY      51103    3820            57650.18  
31            DUBUQUE         52001    4167            55756.18  
77            WEST DES MOINES 50265    2648            52870.35  
              DES MOINES      50321    2561            52844.61  
23            CLINTON         52732    2616            51472.43  
52            CORALVILLE      52241    4677            50517.91  
77            ANKENY          50021    2502            50345.31  
85            AMES            50010    2500            49801.48

In [170]:
zip_stores_margin=sales.groupby(['Zip Code']).agg({'Store Number':'count', 'Margin sum':np.mean})
zip_stores_margin.sort_values('Margin sum', axis=0, ascending=False)

,Store Number,Margin sum
Zip Code,,
50314,6,59501.906667
50320,9,39696.572222
52411,1,37893.050000
50311,7,28313.231429
52401,3,26649.983333
52314,3,23755.700000
51351,2,22749.515000
52807,9,22147.167778
51103,4,17340.965000


In [163]:
sales['Margin sum'].mean()

6942.82343044427

In [172]:
max_city=sales.groupby(['City','Zip Code']).agg({'Store Number':'count', 'Margin sum':np.mean})
max_city.sort_values('Margin sum', axis=0, ascending=False)

,,Store Number,Margin sum
City,Zip Code,,
DES MOINES,50314,6,59501.906667
WINDSOR HEIGHTS,50311,4,40391.362500
DES MOINES,50320,9,39696.572222
CEDAR RAPIDS,52411,1,37893.050000
MOUNT VERNON,52314,2,32910.980000
CEDAR RAPIDS,52401,3,26649.983333
MILFORD,51351,2,22749.515000
DAVENPORT,52807,9,22147.167778
OTTUWMA,52501,1,20674.760000


In [198]:
sales[['Zip Code','Margin sum','Price per Liter mean','Store Number']]

,Zip Code,Margin sum,Price per Liter mean,Store Number
0,50613,103.06,30.775238,2106
1,50613,317.54,25.923611,2106
2,50613,147.42,17.957333,2106
3,50613,134.36,32.468333,2106
4,50613,25.00,20.000000,2106
5,50613,1737.18,29.855932,2106
6,50613,15883.68,16.034769,2106
7,50613,593.46,37.482222,2106
8,50613,8101.20,16.155721,2106
9,50613,262.50,35.000000,2106


In [187]:
pplm=sales.groupby(['Zip Code']).agg({'Store Number':'count','Price per Liter mean':np.mean,
                                      'Margin sum':np.mean})
pplm.sort_values('Price per Liter mean', axis=0, ascending=False)

,Store Number,Price per Liter mean,Margin sum
Zip Code,,,
50846,5,34.148783,433.140000
51553,2,33.961558,95.580000
51534,21,29.849255,486.970952
52338,23,27.983659,557.681304
52084,13,27.012432,984.276154
50312,56,26.957524,795.567500
52223,3,25.066667,24.996667
52401,36,24.939064,2220.831944
50314,93,24.204151,3838.832688


In [189]:
pplm_bottles=sales.groupby(['Zip Code']).agg({'Store Number':'count','Price per Liter mean':np.mean,
                                      'Bottles Sold':np.sum, 'Margin sum':np.sum})
pplm_bottles.sort_values('Price per Liter mean', axis=0, ascending=False)

,Bottles Sold,Store Number,Price per Liter mean,Margin sum
Zip Code,,,,
50846,45,5,34.148783,2165.70
51553,15,2,33.961558,191.16
51534,213,21,29.849255,10226.39
52338,749,23,27.983659,12826.67
52084,353,13,27.012432,12795.59
50312,1554,56,26.957524,44551.78
52223,11,3,25.066667,74.99
52401,1201,36,24.939064,79949.95
50314,5749,93,24.204151,357011.44


# Explore the data

Perform some exploratory statistical analysis and make some plots, such as histograms of transaction totals, bottles sold, etc.

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

## Record your findings

Be sure to write out anything observations from your exploratory analysis.

# Refine the data
Look for any statistical relationships, correlations, or other relevant properties of the dataset.

# Build your models

Using scikit-learn or statsmodels, build the necessary models for your scenario. Evaluate model fit.

In [6]:
from sklearn import linear_model


## Plot your results

Again make sure that you record any valuable information. For example, in the tax scenario, did you find the sales from the first three months of the year to be a good predictor of the total sales for the year? Plot the predictions versus the true values and discuss the successes and limitations of your models

# Present the Results

Present your conclusions and results. If you have more than one interesting model feel free to include more than one along with a discussion. Use your work in this notebook to prepare your write-up.